In [33]:
from rtbm.riemann_theta.riemann_theta import RiemannTheta

import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report

import theano

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Reshape  
from keras.optimizers import SGD

from sklearn.preprocessing import LabelBinarizer

import time

In [36]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:10000,0]
X_train = MNIST_train[0:10000,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

enc = LabelBinarizer()
enc.fit(np.diag([1,1,1,1,1,1,1,1,1,1]))
enc.classes_ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
T=enc.transform(Y_train)

In [60]:
# Keras fit
# 10 sigmoids + 10 softmax
model = Sequential() 

model.add(Dense(10,  input_dim=784))
model.add(Activation('sigmoid'))
model.add(Dense(output_dim=10))
model.add(Activation('softmax'))


sgd = SGD(lr=0.1)

tic = time.clock()

model.compile(loss='mse', optimizer=sgd)

toc = time.clock()

print("Compile time: ",toc-tic)

tic = time.clock()

model.fit(X_train, T, batch_size=1000, nb_epoch=100, validation_data=None, shuffle=False, verbose=1)  
toc = time.clock()

print("Run time: ",toc-tic)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10)`
  import sys


('Compile time: ', 0.003828000000112297)
Epoch 1/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0931
Epoch 2/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0929
Epoch 3/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0928
Epoch 4/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0926
Epoch 5/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0925
Epoch 6/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0924
Epoch 7/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0923
Epoch 8/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0922
Epoch 9/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0921
Epoch 10/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0920
Epoch 11/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0919
Epoch 12/10

10000/10000 [==============================] - 0s 5us/step - loss: 0.0880
Epoch 95/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0880
Epoch 96/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0879
Epoch 97/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0879
Epoch 98/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0879
Epoch 99/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0878
Epoch 100/100
10000/10000 [==============================] - 0s 5us/step - loss: 0.0878
('Run time: ', 19.248299000000088)


In [63]:
# On train set
P=np.argmax(model.predict(X_train),axis=1)

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

          0       0.59      0.78      0.67      1001
          1       0.00      0.00      0.00      1127
          2       1.00      0.00      0.00       991
          3       0.32      0.87      0.47      1032
          4       0.60      0.19      0.29       980
          5       0.16      0.32      0.22       863
          6       0.00      0.00      0.00      1014
          7       0.00      0.00      0.00      1070
          8       0.31      0.48      0.38       944
          9       0.25      0.63      0.36       978

avg / total       0.32      0.32      0.23     10000

[[776   0   0 167   2  33   0   0   2  21]
 [  0   0   0 201   0 391   0   0 516  19]
 [ 70   0   2 375   2  87   0   0 282 173]
 [  9   0   0 893   0  43   0   0  29  58]
 [ 14   0   0 121 189 108   0   0   4 544]
 [118   0   0 346   2 278   0   0  19 100]
 [231   0   0  35   4 499   0   0 104 141]
 [ 10   0   0 102  97 117   0   0  39 705]
 [ 45   0   0 325

In [64]:
# On test set
P=np.argmax(model.predict(X_test),axis=1)

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

             precision    recall  f1-score   support

          0       0.56      0.79      0.65       980
          1       0.00      0.00      0.00      1135
          2       0.62      0.00      0.01      1032
          3       0.31      0.91      0.47      1010
          4       0.62      0.16      0.25       982
          5       0.13      0.22      0.17       892
          6       0.00      0.00      0.00       958
          7       0.00      0.00      0.00      1028
          8       0.28      0.40      0.33       974
          9       0.26      0.64      0.37      1009

avg / total       0.28      0.31      0.22     10000

[[777   0   0 143   3  37   0   0   0  20]
 [  0   0   2 153   0 436   0   0 523  21]
 [ 70   0   5 465   0  75   0   0 281 136]
 [  7   0   1 923   0  36   0   0  16  27]
 [ 16   0   0 141 157 110   0   0   8 550]
 [160   0   0 389   5 199   0   0  30 109]
 [253   0   0  51  11 373   0   0  96 174]
 [ 10   0   0  93  55 104   0   0  42 724]
 [ 75   0   0 356